In [2]:
from glob import glob
import xarray as xr
import pandas as pd
import numpy as np
from pyresample import geometry, create_area_def
from satpy import Scene
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Baseline Combined File

In [11]:
##### This cell is for making a "baseline" combined file, which makes a giant .nc file in the same
#####     format as the original .nc files, with all the data put together in one new file. 

# Here, I'm defining this variable since it's the way I have my file naming system set up. This is convenient
#     for me, since the combined files I'm making are usually for the cruise periods.
period = 'GOM21FA'


# Defining the coordinates of the extent I'll be using.
extent = (-93.2, 27.6, -87.6, 31)

# Defining the path for the file I'll be saving.
savefile = '/home/hboi-ouri/Projects/RS_Files/Task2/Cruise/'+ period +'_MODIS_L2.nc'

# Using glob to get a list of all files I want to combine together.
files = glob('/home/hboi-ouri/Projects/RS_Files/MODISGrids/Cruise/'+ period +'/AQUA_MODIS.*nc')
files.sort()

# To be honest, I'm not super familiar with anything after this point for this cell,
#     and the annotations in here are done by Veronica. If you have any questions about these, 
#     I'd suggest asking her.

# Functions to be used.
def convtoxr(scn, var):
    #convert from dask array
    data = scn[var].load()
    data = data.where(data >= 0)
    data.name = var
    del(data.attrs['_satpy_id']) ### removing, we don't need this.
    data.attrs['area'] = str(data.attrs['area']) ### We may or may not need this in the future so converting it to a string
    return data

def load_satdata(path, 
                 area_deff=None,
                 res=None,
                 name="GOMEX", proj={"proj": "laea"}, area_extent=extent,
                ):
#   get data and attributes
    data = xr.open_dataset(path, group='geophysical_data')
    navigation = xr.open_dataset(path, group='navigation_data')
    navigation = navigation.rename({'pixel_control_points': 'pixels_per_line'})
    attrs = xr.open_dataset(path).attrs
    
#   area definition if not defined
    if res is None:
        # get the resolution of the Sat product
        val, unit = attrs['spatialResolution'].split()
        res  = xr.DataArray(float(val), attrs={"units": unit})
    if area_deff is None:
        area_deff = create_area_def(name, proj, area_extent=area_extent, units="degrees", resolution=res)
        
#   reprojecting
    scn = Scene()
    swath_def = geometry.SwathDefinition(lons=navigation.longitude, lats= navigation.latitude)
    vnames = [v for v in data.data_vars if 'Rrs' in v]
    for var in vnames:
        scn[var] = data[var]
        scn[var].attrs["area"] = swath_def
    resampled_scn = scn.resample(area_deff, radius_of_influence=int(res))
    resampled_scn.compute()
    lons, lats = resampled_scn[vnames[0]].attrs['area'].get_lonlats()
#   Converting back to xarray dataset
    ds = xr.merge([convtoxr(resampled_scn, var) for var in vnames])
    ds.attrs = attrs
    ds = ds.assign_coords({
                            'lon': (('y', 'x'), lons),
                            'lat': (('y', 'x'), lats),
                            'time': pd.to_datetime(attrs['time_coverage_start'])
                            })
#   getting rid of the crs coord
    ds = ds.drop('crs')
    print(str(ds.time.data))
    return ds

# Area setup for GOMEX
resval = 1000 # resolution of the sat product (300 m for Sentinel and MERIS, 1 km for MODIS)
res = xr.DataArray(resval, attrs={"units": "meters"})
lat_0, lon_0 = 28.5, -90.8 # just some rounded up mean from the lat lon range of the sat product
proj = {'proj': 'laea', 'lat_0': lat_0, 'lon_0': lon_0, 'a': 6371228.0, 'units': 'm'} # projection object
area = create_area_def("GOMEX", proj,
                       area_extent=extent, units="degrees", 
                       resolution=res
                      ) # creating a fixed area to reproject to

# Reprojectig files
dsets = [load_satdata(f, area_deff=area, res=res) for f in files] # looping trough the files
combined = xr.concat(dsets, dim='time') #combining in one file


#save combined file
# give a sensible name i.e. cruiseID_satID.nc for every cruise and satelite
combined.to_netcdf(savefile) 

    
    

Rounding shape to (380, 542) and resolution from (1000.0, 1000.0) meters to (999.0798276127949, 999.6147938626385) meters
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when conver

2021-10-22T18:15:00.881000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

2021-10-22T19:55:00.913000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

...

2021-11-12T18:35:01.458000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

2021-11-12T20:10:01.636000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

2021-11-13T19:15:01.050000000


# Adding Variables to Baseline Combined File

In [18]:
# Here, I'm defining functions for all the variables I want in my combined file. The ones in this cell are 
#     from the algorithms we're using for CDOM and DOC. Depending on what variables you chose to include when
#     you downloaded the files, you can mess around with a lot of different things.

# "lambd" is defined just for the sake of the algorithm, since the algorithm has calculations for 
#     both 300 nm and 350 nm wavelengths. It isn't actually used here, so don't worry about it.
def get_CDOM300(ds, lambd=300):

        # This excludes all data that are 0 or below. I'm not sure if any data can be negative, but 
        #     just in case it can, this prevents it from breaking the algorithm.
        ds=ds.where(ds>0)
        
        
        # The stuff here is for the algorithm, defining certain coefficients.
        alpha = -.0206
        beta = -.6128
        gamma = -.007
        delta = -.4944
        epsilon = .9362
        zeta = .9666

        # This is the actual equation we're using to get aCDOM300. 
        aCDOM300 = np.exp(alpha * np.log(ds.Rrs_443.where(ds.Rrs_443>=0)) +
                      beta * np.log(ds.Rrs_488.where(ds.Rrs_488>=0)) +
                      gamma * np.log(ds.Rrs_531.where(ds.Rrs_531>=0)) +
                      delta * np.log(ds.Rrs_555.where(ds.Rrs_555>=0)) +
                      epsilon * np.log(ds.Rrs_667.where(ds.Rrs_667>=0)) + zeta)

        return aCDOM300
    
    
def get_CDOM355(ds, lambd=355):
        
        ds=ds.where(ds>0)

        alpha = .0376
        beta = -.8714
        gamma = -.0352
        delta = -.2739
        epsilon = .9591
        zeta = -.1071

        aCDOM355 = np.exp(alpha * np.log(ds.Rrs_443.where(ds.Rrs_443>=0)) +
                      beta * np.log(ds.Rrs_488.where(ds.Rrs_488>=0)) +
                      gamma * np.log(ds.Rrs_531.where(ds.Rrs_531>=0)) +
                      delta * np.log(ds.Rrs_555.where(ds.Rrs_555>=0)) +
                      epsilon * np.log(ds.Rrs_667.where(ds.Rrs_667>=0)) + zeta)

        return aCDOM355
    
    
def get_S275_295(ds):
        
        ds=ds.where(ds>0)

        alpha = -.0537
        beta = .2689
        gamma = .1017
        delta = -.2097
        epsilon = -.0893
        zeta = -3.6853

        S275_295 = np.exp(alpha * np.log(ds.Rrs_443.where(ds.Rrs_443>=0)) +
                      beta * np.log(ds.Rrs_488.where(ds.Rrs_488>=0)) +
                      gamma * np.log(ds.Rrs_531.where(ds.Rrs_531>=0)) +
                      delta * np.log(ds.Rrs_555.where(ds.Rrs_555>=0)) +
                      epsilon * np.log(ds.Rrs_667.where(ds.Rrs_667>=0)) + zeta)

        return S275_295


def get_DOC(ds):
        
        ds=ds.where(ds>0)
        
        alpha = -.0206
        beta = -.6128
        gamma = -.007
        delta = -.4944
        epsilon = .9362
        zeta = .9666

        old_aCDOM300 = np.exp(alpha * np.log(ds.Rrs_443.where(ds.Rrs_443>=0)) +
                      beta * np.log(ds.Rrs_488.where(ds.Rrs_488>=0)) +
                      gamma * np.log(ds.Rrs_531.where(ds.Rrs_531>=0)) +
                      delta * np.log(ds.Rrs_555.where(ds.Rrs_555>=0)) +
                      epsilon * np.log(ds.Rrs_667.where(ds.Rrs_667>=0)) + zeta)
        
        alpha = -.0537
        beta = .2689
        gamma = .1017
        delta = -.2097
        epsilon = -.0893
        zeta = -3.6853

        S275_295 = np.exp(alpha * np.log(ds.Rrs_443.where(ds.Rrs_443>=0)) +
                      beta * np.log(ds.Rrs_488.where(ds.Rrs_488>=0)) +
                      gamma * np.log(ds.Rrs_531.where(ds.Rrs_531>=0)) +
                      delta * np.log(ds.Rrs_555.where(ds.Rrs_555>=0)) +
                      epsilon * np.log(ds.Rrs_667.where(ds.Rrs_667>=0)) + zeta)
        
        DOC = (old_aCDOM300)/(np.exp(-15.05 - 33.95 * S275_295) + 
                            np.exp(-1.502 - 104.3 * S275_295))

        return DOC
    
    



In [5]:
#### To save files. This allows me to put the variables I defined in the previous functions into the 
####    baseline combined files we made earlier.

# Creating a list of combined files we're putting new variables into
combfiles = glob('/home/hboi-ouri/Projects/RS_Files/Task4/*_MODIS_L2.nc')
combfiles.sort()

for file in combfiles:

    # This is just for the sake of naming the file later.
    month = file.split('/')[6][0:7]
        
    with xr.open_dataset(file) as ds:
        # Here, we're loading the dataset and putting the new variables into the combiend file
        data = ds.load()
        data['aCDOM300'] = get_CDOM300(ds)
        data['aCDOM355'] = get_CDOM355(ds)
        data['S275_295'] = get_S275_295(ds)
        data['DOC'] = get_DOC(ds)
        data['aCDOM300_special'] = get_aCDOM300_special(ds)
        
    
        # Saving the file
        data.to_netcdf('/home/hboi-ouri/Projects/RS_Files/Task4/'+ month +'_MODIS_L2_aCDOM.nc')

    